In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv('train.csv', encoding='utf8')
len(df)

3589918

In [2]:
df_alone = df.copy()

Для простоты переведем 'quarter' в числа year * 3 + quarter_num

In [3]:
def quarter_to_num(txt):
    arr = txt.split('Q')
    return int(arr[0]) * 4 + int(arr[1])

def date_to_month_num(txt):
    if txt != txt:
        return None
    arr = txt.split('-')
    return int(arr[0]) * 12 + int(arr[1])

df_alone['quarter'] = df_alone['quarter'].apply(lambda x: quarter_to_num(x))

Нам достаточно признаков о клиенте по клиенту, не будем учитывать его порядковый номер, как и порядковый номер заказа, postal_coвe - почти везде None

In [4]:
df_alone = df_alone.drop(columns=['client_id', 'npo_account_id', 'postal_code',])

Как видно slctn_nmbr это номер выборки из которой пришли эти данные, это может быть значимым моментом, поэотму закодируем их OneHot

In [5]:
df_alone = pd.get_dummies(df_alone, columns=['slctn_nmbr'], prefix='slctn_nmbr', drop_first=True)

Загрузим уровень инфляции в РФ за каждый месяц

In [6]:
infl = {'2023-01': 11.76, '2023-02': 10.97, '2023-03': 3.51, '2023-04': 2.3, '2023-05': 2.5, '2023-06': 3.24, '2023-07': 4.3, '2023-08': 5.13, '2023-09': 6.0, '2023-10': 6.68, '2023-11': 7.47, '2023-12': 7.42, '2023-13': 7.42, '2022-01': 8.74, '2022-02': 9.16, '2022-03': 16.7, '2022-04': 17.83, '2022-05': 17.11, '2022-06': 15.9, '2022-07': 15.09, '2022-08': 14.3, '2022-09': 13.67, '2022-10': 12.63, '2022-11': 11.97, '2022-12': 11.92, '2022-13': 11.92, '2021-01': 5.19, '2021-02': 5.67, '2021-03': 5.78, '2021-04': 5.52, '2021-05': 6.01, '2021-06': 6.51, '2021-07': 6.47, '2021-08': 6.69, '2021-09': 7.41, '2021-10': 8.14, '2021-11': 8.4, '2021-12': 8.39, '2021-13': 8.39, '2020-01': 2.42, '2020-02': 2.31, '2020-03': 2.55, '2020-04': 3.1, '2020-05': 3.03, '2020-06': 3.21, '2020-07': 3.37, '2020-08': 3.57, '2020-09': 3.67, '2020-10': 3.98, '2020-11': 4.42, '2020-12': 4.91, '2020-13': 4.91, '2019-01': 5.0, '2019-02': 5.24, '2019-03': 5.27, '2019-04': 5.17, '2019-05': 5.13, '2019-06': 4.66, '2019-07': 4.59, '2019-08': 4.33, '2019-09': 3.99, '2019-10': 3.77, '2019-11': 3.54, '2019-12': 3.05, '2019-13': 3.05, '2018-01': 2.21, '2018-02': 2.2, '2018-03': 2.36, '2018-04': 2.41, '2018-05': 2.42, '2018-06': 2.3, '2018-07': 2.5, '2018-08': 3.07, '2018-09': 3.39, '2018-10': 3.55, '2018-11': 3.83, '2018-12': 4.27, '2018-13': 4.27, '2017-01': 5.02, '2017-02': 4.59, '2017-03': 4.25, '2017-04': 4.13, '2017-05': 4.09, '2017-06': 4.35, '2017-07': 3.86, '2017-08': 3.29, '2017-09': 2.96, '2017-10': 2.73, '2017-11': 2.5, '2017-12': 2.52, '2017-13': 2.52, '2016-01': 9.77, '2016-02': 8.06, '2016-03': 7.26, '2016-04': 7.24, '2016-05': 7.3, '2016-06': 7.48, '2016-07': 7.21, '2016-08': 6.84, '2016-09': 6.42, '2016-10': 6.09, '2016-11': 5.76, '2016-12': 5.38, '2016-13': 5.38, '2015-01': 14.97, '2015-02': 16.71, '2015-03': 16.93, '2015-04': 16.42, '2015-05': 15.78, '2015-06': 15.29, '2015-07': 15.64, '2015-08': 15.77, '2015-09': 15.68, '2015-10': 15.59, '2015-11': 14.98, '2015-12': 12.91, '2015-13': 12.91, '2014-01': 6.05, '2014-02': 6.2, '2014-03': 6.92, '2014-04': 7.33, '2014-05': 7.59, '2014-06': 7.8, '2014-07': 7.45, '2014-08': 7.56, '2014-09': 8.03, '2014-10': 8.3, '2014-11': 9.07, '2014-12': 11.36, '2014-13': 11.36, '2013-01': 7.07, '2013-02': 7.28, '2013-03': 7.02, '2013-04': 7.23, '2013-05': 7.38, '2013-06': 6.88, '2013-07': 6.45, '2013-08': 6.49, '2013-09': 6.13, '2013-10': 6.25, '2013-11': 6.48, '2013-12': 6.45, '2013-13': 6.45, '2012-01': 4.16, '2012-02': 3.74, '2012-03': 3.7, '2012-04': 3.57, '2012-05': 3.61, '2012-06': 4.3, '2012-07': 5.59, '2012-08': 5.95, '2012-09': 6.58, '2012-10': 6.55, '2012-11': 6.47, '2012-12': 6.58, '2012-13': 6.58, '2011-01': 9.56, '2011-02': 9.47, '2011-03': 9.46, '2011-04': 9.61, '2011-05': 9.59, '2011-06': 9.42, '2011-07': 9.01, '2011-08': 8.16, '2011-09': 7.21, '2011-10': 7.19, '2011-11': 6.78, '2011-12': 6.1, '2011-13': 6.1, '2010-01': 8.02, '2010-02': 7.18, '2010-03': 6.46, '2010-04': 6.04, '2010-05': 5.97, '2010-06': 5.74, '2010-07': 5.46, '2010-08': 6.04, '2010-09': 6.96, '2010-10': 7.5, '2010-11': 8.06, '2010-12': 8.78, '2010-13': 8.78, '2009-01': 13.35, '2009-02': 13.85, '2009-03': 13.98, '2009-04': 13.16, '2009-05': 12.28, '2009-06': 11.87, '2009-07': 12.01, '2009-08': 11.6, '2009-09': 10.69, '2009-10': 9.69, '2009-11': 9.1, '2009-12': 8.8, '2009-13': 8.8, '2008-01': 12.56, '2008-02': 12.66, '2008-03': 13.35, '2008-04': 14.3, '2008-05': 15.12, '2008-06': 15.14, '2008-07': 14.73, '2008-08': 15.04, '2008-09': 15.05, '2008-10': 14.23, '2008-11': 13.78, '2008-12': 13.28, '2008-13': 13.28, '2007-01': 8.2, '2007-02': 7.61, '2007-03': 7.37, '2007-04': 7.6, '2007-05': 7.76, '2007-06': 8.48, '2007-07': 8.7, '2007-08': 8.59, '2007-09': 9.35, '2007-10': 10.83, '2007-11': 11.49, '2007-12': 11.87, '2007-13': 11.87, '2006-01': 10.71, '2006-02': 11.18, '2006-03': 10.61, '2006-04': 9.77, '2006-05': 9.42, '2006-06': 9.03, '2006-07': 9.26, '2006-08': 9.62, '2006-09': 9.44, '2006-10': 9.15, '2006-11': 9.03, '2006-12': 9.0, '2006-13': 9.0, '2005-01': 12.7, '2005-02': 12.96, '2005-03': 13.63, '2005-04': 13.77, '2005-05': 13.84, '2005-06': 13.68, '2005-07': 13.16, '2005-08': 12.53, '2005-09': 12.33, '2005-10': 11.68, '2005-11': 11.27, '2005-12': 10.91, '2005-13': 10.91, '2004-01': 11.28, '2004-02': 10.58, '2004-03': 10.25, '2004-04': 10.22, '2004-05': 10.15, '2004-06': 10.13, '2004-07': 10.36, '2004-08': 11.28, '2004-09': 11.38, '2004-10': 11.53, '2004-11': 11.7, '2004-12': 11.74, '2004-13': 11.74, '2003-01': 14.29, '2003-02': 14.82, '2003-03': 14.78, '2003-04': 14.62, '2003-05': 13.62, '2003-06': 13.93, '2003-07': 13.91, '2003-08': 13.35, '2003-09': 13.28, '2003-10': 13.2, '2003-11': 12.48, '2003-12': 11.99, '2003-13': 11.99, '2002-01': 18.96, '2002-02': 17.66, '2002-03': 16.76, '2002-04': 16.04, '2002-05': 15.94, '2002-06': 14.69, '2002-07': 15.0, '2002-08': 15.09, '2002-09': 14.86, '2002-10': 14.84, '2002-11': 15.12, '2002-12': 15.06, '2002-13': 15.06, '2001-01': 20.71, '2001-02': 22.19, '2001-03': 23.67, '2001-04': 24.77, '2001-05': 24.81, '2001-06': 23.68, '2001-07': 22.05, '2001-08': 20.88, '2001-09': 20.02, '2001-10': 18.82, '2001-11': 18.63, '2001-12': 18.58, '2001-13': 18.58, '2000-01': 28.93, '2000-02': 25.11, '2000-03': 22.49, '2000-04': 19.95, '2000-05': 19.4, '2000-06': 20.15, '2000-07': 18.94, '2000-08': 18.73, '2000-09': 18.54, '2000-10': 19.41, '2000-11': 19.75, '2000-12': 20.2, '2000-13': 20.2, '1999-01': 96.92, '1999-02': 103.24, '1999-03': 107.58, '1999-04': 113.06, '1999-05': 116.71, '1999-06': 120.67, '1999-07': 126.51, '1999-08': 121.03, '1999-09': 62.03, '1999-10': 57.12, '1999-11': 50.52, '1999-12': 36.56, '1999-13': 36.56, '1998-01': 10.13, '1998-02': 9.43, '1998-03': 8.57, '1998-04': 7.95, '1998-05': 7.48, '1998-06': 6.4, '1998-07': 5.59, '1998-08': 9.62, '1998-09': 52.21, '1998-10': 58.85, '1998-11': 66.84, '1998-12': 84.44, '1998-13': 84.44, '1997-01': 19.74, '1997-02': 18.28, '1997-03': 16.71, '1997-04': 15.34, '1997-05': 14.59, '1997-06': 14.51, '1997-07': 14.75, '1997-08': 14.83, '1997-09': 14.11, '1997-10': 12.95, '1997-11': 11.54, '1997-12': 11.03, '1997-13': 11.03, '1996-01': 104.5, '1996-02': 89.34, '1996-03': 78.66, '1996-04': 68.27, '1996-05': 58.4, '1996-06': 50.25, '1996-07': 43.6, '1996-08': 37.05, '1996-09': 31.63, '1996-10': 27.21, '1996-11': 23.95, '1996-12': 21.81, '1996-13': 21.81, '1995-01': 214.68, '1995-02': 215.24, '1995-03': 219.73, '1995-04': 219.68, '1995-05': 222.73, '1995-06': 224.73, '1995-07': 224.89, '1995-08': 224.7, '1995-09': 214.18, '1995-10': 186.09, '1995-11': 161.0, '1995-12': 131.33, '1995-13': 131.33, '1994-01': 780.85, '1994-02': 682.8, '1994-03': 600.09, '1994-04': 539.87, '1994-05': 479.24, '1994-06': 412.09, '1994-07': 340.71, '1994-08': 265.93, '1994-09': 221.19, '1994-10': 209.09, '1994-11': 204.36, '1994-12': 215.02, '1994-13': 215.02, '1993-01': 850.46, '1993-02': 758.85, '1993-03': 694.06, '1993-04': 674.49, '1993-05': 717.4, '1993-06': 722.89, '1993-07': 810.61, '1993-08': 956.51, '1993-09': 1065.47, '1993-10': 1033.23, '1993-11': 945.97, '1993-12': 839.87, '1993-13': 839.87, 'nan': 0}

Ну и теперь рассмотрим дату прихода и ухода клиента как значение инфляции в этом месяце

In [7]:
def only_month(x):
    if x != x:
        return "nan"
    xarr = x.split("-")
    return xarr[0] + "-" + xarr[1]

df_alone['frst_pmnt_date'] = df_alone["frst_pmnt_date"].apply(lambda x: only_month(x))
df_alone['lst_pmnt_date_per_qrtr'] = df_alone["lst_pmnt_date_per_qrtr"].apply(lambda x: only_month(x))

In [8]:
df_alone["frst_pmnt_date"] = df_alone["frst_pmnt_date"].replace(infl)
df_alone["lst_pmnt_date_per_qrtr"] = df_alone["lst_pmnt_date_per_qrtr"].replace(infl)

Тут блок со всеми фичами, кроме region

Изначально мы брали только пулл числовых признаков и экспериментировали на них, может на некоторых делали OneHot или Label

Постепенно добавляя признаки пришли все кроме region

region кодировался MeanTarget, но это не давало пользы

Разделение на тренировочную и валидационную

In [13]:
X = df_alone.drop(['churn', 'region'], axis=1)
y = df_alone['churn']

In [14]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=778)

Тесты с нормировками и калибровкой

In [27]:
# from sklearn.pipeline import Pipeline
# from sklearn.compose import ColumnTransformer
# from sklearn.preprocessing import StandardScaler

# preprocessor = ColumnTransformer(
#     transformers=[
#         ('num', StandardScaler(), num_features),
#         # ('cat', OneHotEncoder(drop='first'), categorical_cols)
#     ])

# pipeline = Pipeline(steps=[('preprocessor', preprocessor)])
# X_train_enc = pipeline.fit_transform(X_train_enc)
# X_val_enc = pipeline.transform(X_val_enc)

In [32]:
# from imblearn.over_sampling import SMOTE

# smote = SMOTE(random_state=42)
# X_train_enc, y_train = smote.fit_resample(X_train_enc, y_train)

Обучение модели с подобранными гиперпараметрами

In [15]:
from catboost import CatBoostClassifier
from sklearn.metrics import f1_score
from sklearn.model_selection import GridSearchCV

model = CatBoostClassifier(
    iterations=10000,
    learning_rate=0.1,
    loss_function='Logloss',
    eval_metric='F1',
    use_best_model=True,
)

model.fit(X_train, y_train, eval_set=(X_val, y_val), verbose=100)

predictions = model.predict(X_val)

f1 = f1_score(y_val, predictions)
print(f"F1-score: {f1}")

0:	learn: 0.4230924	test: 0.4261288	best: 0.4261288 (0)	total: 272ms	remaining: 45m 19s
100:	learn: 0.6479089	test: 0.6443152	best: 0.6443152 (100)	total: 11.8s	remaining: 19m 13s
200:	learn: 0.7036957	test: 0.7006619	best: 0.7006619 (200)	total: 23.3s	remaining: 18m 57s
300:	learn: 0.7255879	test: 0.7217273	best: 0.7217273 (300)	total: 34.5s	remaining: 18m 33s
400:	learn: 0.7392750	test: 0.7350418	best: 0.7350418 (400)	total: 45.8s	remaining: 18m 16s
500:	learn: 0.7489578	test: 0.7438085	best: 0.7438469 (496)	total: 57.1s	remaining: 18m 3s
600:	learn: 0.7551541	test: 0.7496267	best: 0.7496267 (600)	total: 1m 8s	remaining: 17m 48s
700:	learn: 0.7597299	test: 0.7528892	best: 0.7529158 (696)	total: 1m 19s	remaining: 17m 32s
800:	learn: 0.7651900	test: 0.7562638	best: 0.7562638 (800)	total: 1m 30s	remaining: 17m 19s
900:	learn: 0.7685516	test: 0.7580390	best: 0.7582822 (892)	total: 1m 41s	remaining: 17m 5s
1000:	learn: 0.7725016	test: 0.7604124	best: 0.7610448 (986)	total: 1m 52s	remainin

Видим F1-score и сохраняем модель

In [16]:
model.save_model('catboost_model.cbm', format='cbm')

## Часть TEST (тоже самое, что и в project-приложении)

In [17]:
import pandas as pd
import numpy as np

df = pd.read_csv('test.csv', encoding='utf8')
len(df)

633434

In [18]:
df_alone = df.copy()

Делаем ту же предобработку

In [19]:
def quarter_to_num(txt):
    arr = txt.split('Q')
    return int(arr[0]) * 4 + int(arr[1])

df_alone['quarter'] = df_alone['quarter'].apply(lambda x: quarter_to_num(x))

In [20]:
df_alone = df_alone.drop(columns=['client_id', 'npo_account_id', 'postal_code',])

In [21]:
df_alone = pd.get_dummies(df_alone, columns=['slctn_nmbr'], prefix='slctn_nmbr', drop_first=True)

In [22]:
def only_month(x):
    if x != x:
        return "nan"
    xarr = x.split("-")
    return xarr[0] + "-" + xarr[1]

df_alone['frst_pmnt_date'] = df_alone["frst_pmnt_date"].apply(lambda x: only_month(x))
df_alone['lst_pmnt_date_per_qrtr'] = df_alone["lst_pmnt_date_per_qrtr"].apply(lambda x: only_month(x))

In [23]:
df_alone["frst_pmnt_date"] = df_alone["frst_pmnt_date"].replace(infl)
df_alone["lst_pmnt_date_per_qrtr"] = df_alone["lst_pmnt_date_per_qrtr"].replace(infl)

In [24]:
X = df_alone.drop(['region'], axis=1)

In [25]:
test_pred = model.predict(X)

Делаем inference

In [26]:
df['churn'] = test_pred

In [27]:
df_test = df[['npo_account_id', 'quarter', 'churn']]

In [28]:
df_test

,npo_account_id,quarter,churn
0,0xAA9E8DDF1310724995598EA2B42D7D87,2017Q2,0
1,0x3B9B09857D152F468A42C5DEE6D723F7,2021Q4,0
2,0xA7F12C768A4FB38311E835E4A7632E00,2015Q3,0
3,0xBFE4BDFBE0E037478C29BA4F38121B15,2015Q3,0
4,0x4B5C7804E403C842B4ECBF69BC3EDD7C,2011Q3,0
...,...,...,...
633429,0x3086DA482BB98F4CB01FBFBCF699B603,2022Q3,0
633430,0x44CA6910C3C5904BA87936C53926BABC,2020Q2,0
633431,0xA62A005056964A2C11ED7C7CECEF8EC0,2022Q4,0
633432,0xABCE12A6F911374E96C7CB080F45C900,2021Q2,0


Сохраняем

In [151]:
df_test.to_csv('try3.csv', index=False)